In [2]:
import pandas as pd
import os
import warnings
import pymongo
import json
import datetime

pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore', category=RuntimeWarning)

for i in range(1): # one year only
    year = 2007
    print("YEAR: ", year)
    
    file_path = f"results/inputs_csv_zscore_last/PrixCarburants_annuel_filtered_{year}.csv"
    
    if not os.path.exists(file_path):
        print(f"Le fichier {file_path} n'existe pas.")
    else:
        df = pd.read_csv(file_path)
        df['date2'] = df['date']
        df['date'] = pd.to_datetime(df['date'], format='%Y_%m_%d')
        df['heuremin'] = pd.to_datetime(df['heuremin'], format='%H:%M').dt.strftime('%H:%M:%S')
        print(df.head(5))
        print(df.dtypes)
        
        # Setup MongoDB connection (local)
        mongo_host = "mongodb://localhost"
        mongo_port = 27017
        client_mongo = pymongo.MongoClient(host=mongo_host, port=mongo_port)
        
        db_mongo = client_mongo.get_database("gas_station_price")
        collection_mongo = db_mongo.get_collection("gas_station_price")
        
        records = df.to_dict(orient="records")
        collection_mongo.insert_many(records)
        print("correctly loaded", year, "csv to MongoDB")

YEAR:  2007
        date     nom   latitude  longitude    cp                  ville  \
0 2007-01-04  Gazole  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
1 2007-01-04    SP95  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
2 2007-01-09  Gazole  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
3 2007-01-09    SP95  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   
4 2007-01-12  Gazole  4620114.0   519791.0  1000  SAINT-DENIS-LèS-BOURG   

           adresse  valeur  heuremin  gas_station_id       date2  
0  ROUTE NATIONALE   999.0  18:43:00         1000001  2007_01_04  
1  ROUTE NATIONALE  1170.0  18:43:00         1000001  2007_01_04  
2  ROUTE NATIONALE   999.0  08:32:00         1000001  2007_01_09  
3  ROUTE NATIONALE  1160.0  08:32:00         1000001  2007_01_09  
4  ROUTE NATIONALE   989.0  08:42:00         1000001  2007_01_12  
date              datetime64[ns]
nom                       object
latitude                 float64
longitude                float64
cp 